In [22]:
from datetime import datetime
import json
import pandas as pd

def get_poster(movie):
    if movie["poster_path"] is not None:
        return location + movie["poster_path"]
    elif movie["backdrop_path"] is not None:
        return location + movie["backdrop_path"]
    else:
        return None

def set_dates(date):
    if type(date) != str or date == "":
        return None
    elif '/' in date:
        return datetime.strptime(date, '%d/%m/%Y')
    elif '-' in date:
        return datetime.strptime(date, '%Y-%m-%d')
    else:
        return None
    
def set_countries(country):
    if type(country) != str or '{' not in country:
        return None
    else:
        return country.replace("iso_3166_1", "id")
    
path = 'data/movies_metadata.csv'
movies = pd.read_csv(path, low_memory=False)

cols = ["id", "title", "belongs_to_collection", "budget", "genres", 
        "overview", "production_companies", "production_countries",
       "release_date", "revenue", "runtime", "tagline", ]
movies = movies[cols]

movies = movies.loc[movies.belongs_to_collection.notnull()]

movies["production_countries"] = movies["production_countries"].apply(lambda x: set_countries(x))

list_cols = ["genres", "production_companies", 
            "production_countries"]
for col in list_cols:
    movies = movies[movies[col].str.contains("{") == True]
    movies[col] = movies[col].apply(lambda x: list(eval(x)))
    
movies["belongs_to_collection"] = movies["belongs_to_collection"].apply(lambda x: eval(x))
location = "https://image.tmdb.org/t/p/original"    
movies["poster"] = movies["belongs_to_collection"].apply(lambda x: get_poster(x))
movies = movies.loc[movies.poster.notnull()]
movies.drop("belongs_to_collection", axis='columns', inplace=True)
                                         
movies["release_date"] = movies["release_date"].apply(lambda x: set_dates(x))

movies["id"] = movies["id"].astype(int)
movies["budget"] = movies["budget"].astype(float)
movies["revenue"] = movies["revenue"].astype(int)

#movies["release_date"] = movies["release_date"].apply(lambda x: None if type(x) == pd.NaT else x)
movies.replace({pd.NaT: None}, inplace=True)

movies[movies["id"] == 23397]

,id,title,budget,genres,overview,production_companies,production_countries,release_date,revenue,runtime,tagline,poster


In [17]:
#Genres, Countries, Companies
def json_to_table(movies, column_name): 
    entities = {}
    entitities_values = set()
    for line in movies[column_name]:
        for x in line:
            key = x["id"]
            value = x["name"]
            if value not in entitities_values:
                entitities_values.add(value)
                entities[key] = value
    return entities     
    
    
geners = json_to_table(movies, "genres")
companies = json_to_table(movies, "production_companies")
countries = json_to_table(movies, "production_countries")
companies[23397]

KeyError: 23397

In [21]:
#MoviesGenres, MoviesCountries, MoviesCompanies
def relative_data(movies, col):
    data = []
    for i, movie in movies.iterrows():
        movie_id = int(movie.id)
        for z in movie[col]:
            data.append((movie_id, z["id"]))
    return data

movies_genres = relative_data(movies, "genres")
movies_companies = relative_data(movies, "production_companies")
movies_countries = relative_data(movies, "production_countries")
#d = {x: y for x, y in movies_companies}
print(d)

{862: 3, 15602: 19464, 11862: 9195, 710: 7576, 21032: 4105, 9273: 10210, 8012: 8411, 9598: 2537, 9312: 4174, 10530: 10217, 11525: 12, 10634: 12, 755: 53009, 9102: 23098, 10874: 6254, 27793: 53009, 9737: 10288, 414: 31080, 5894: 30334, 8839: 11098, 8068: 11705, 1572: 1504, 13552: 18902, 9073: 6194, 9070: 9255, 1642: 5, 11980: 7405, 10802: 14723, 10149: 14, 9348: 8411, 3512: 63906, 50797: 6454, 76: 6194, 2292: 16934, 8467: 11061, 15789: 2, 628: 360, 11: 306, 63105: 21437, 18220: 17068, 110: 1245, 108: 1610, 109: 850, 22586: 4108, 11395: 915, 9059: 69626, 193: 4, 3049: 10210, 9331: 2767, 9495: 6455, 888: 1353, 8587: 10217, 11596: 12, 854: 552, 36593: 4, 1637: 306, 8831: 4248, 11667: 7220, 8011: 30655, 2758: 258, 9409: 9195, 306: 30, 11310: 97, 1634: 23397, 5503: 6194, 2019: 11462, 9255: 11751, 329: 56, 11231: 441, 5550: 41, 11236: 6194, 8850: 33, 78: 7965, 9066: 4, 771: 477, 812: 2, 280: 61409, 268: 6194, 274: 55072, 10020: 10282, 10824: 1382, 25059: 2, 11827: 4952, 8766: 15343, 19042: 84

In [4]:
def to_peewee_compatible(parsed_json):
    return [
        {"id": key, 'name': value}
        for key, value
        in parsed_json.items()
        #in json_to_table(movies, column_name).items()
    ]


def get_all_from_table(table):
    for category in table.select():
        print(category)

def insert_parsed_json(db, parsed_json, table_name):
    to_insert = to_peewee_compatible(parsed_json)
    table = get_table_class(table_name)
    for var in to_insert:
        table.insert(var).on_conflict_ignore().execute()
    db.commit()
    #get_all_from_table(table)

        
def get_table_class(table_name):
    name = table_name.title()
    tables = (t for t in TABLES if t.__name__ == name)
    return next(tables, None)

In [ ]:
movies["vote_average"] = 0
movies["vote_count"] = 0
cols = ["id", "title", "budget", "overview", "release_date",
        "revenue", "runtime", "tagline", "poster", "vote_average",
       "vote_count"]
table = get_table_class("movies")
to_insert_movies = movies[cols].to_dict('records')
for line in to_insert_movies:
    table.insert(line).on_conflict_ignore().execute()
database.commit()

In [ ]:
genres = json_to_table(movies, "genres")
companies = json_to_table(movies, "production_companies")
countries = json_to_table(movies, "production_countries")
insert_parsed_json(database, genres, "genres")
insert_parsed_json(database, companies, "companies")
insert_parsed_json(database, countries, "countries")

In [25]:
parsed_movies_genres = [{"movie_id": x, "genre_id": y} for x, y in movies_genres]
parsed_movies_companies = [{"movie_id": x, "company_id": y} for x, y in movies_companies]
parsed_movies_countries = [{"movie_id": x, "country_id": y} for x, y in movies_countries]

# insert_rules = [
#     (MoviesGenres, parsed_movies_genres),
#     (MoviesCompanies, parsed_movies_companies),
#     (MoviesCountries, parsed_movies_countries)
# ]
# for table, entities in insert_rules:
#     #table.insert_many(entities).on_conflict_ignore().execute()
#     for entity in entities:
#         #table = get_table_class(table_name)
#         #print(entity)
#         table.insert(entity).on_conflict_ignore().execute()
# database.commit()
[x for x in parsed_movies_companies x["movie_id"]]

[{'movie_id': 862, 'company_id': 3},
 {'movie_id': 15602, 'company_id': 6194},
 {'movie_id': 15602, 'company_id': 19464},
 {'movie_id': 11862, 'company_id': 5842},
 {'movie_id': 11862, 'company_id': 9195},
 {'movie_id': 710, 'company_id': 60},
 {'movie_id': 710, 'company_id': 7576},
 {'movie_id': 21032, 'company_id': 33},
 {'movie_id': 21032, 'company_id': 56},
 {'movie_id': 21032, 'company_id': 4105},
 {'movie_id': 9273, 'company_id': 5682},
 {'movie_id': 9273, 'company_id': 6194},
 {'movie_id': 9273, 'company_id': 10210},
 {'movie_id': 8012, 'company_id': 216},
 {'movie_id': 8012, 'company_id': 8411},
 {'movie_id': 9598, 'company_id': 33},
 {'movie_id': 9598, 'company_id': 2537},
 {'movie_id': 9312, 'company_id': 12},
 {'movie_id': 9312, 'company_id': 4174},
 {'movie_id': 10530, 'company_id': 2},
 {'movie_id': 10530, 'company_id': 10217},
 {'movie_id': 11525, 'company_id': 12},
 {'movie_id': 10634, 'company_id': 12},
 {'movie_id': 755, 'company_id': 59},
 {'movie_id': 755, 'company_i

In [32]:
from peewee import (
    DateField, FloatField, ForeignKeyField, IntegerField, Model,
    PostgresqlDatabase, TextField,
)

database = PostgresqlDatabase(
    "d9jhv03hhlvm72",
    user="zkbbhlzsczfcvq",
    password="d32d84caf34356f4f116301920289fdcf5e73bd41567372fecfb65d0341cade9",
    host="ec2-35-172-246-19.compute-1.amazonaws.com",
    port=5432,
)


#movies_db_csv = "C:\Users\polla\pythoncourse\week14\project2\movies.db"
#database = SqliteDatabase("movies.db")


class UnknownField(object):
    def __init__(self, *_, **__):
        pass


class BaseModel(Model):
    class Meta:
        database = database

        
class Genres(BaseModel):
    id = IntegerField(unique=True)
    name = TextField()

    class Meta:
        table_name = 'genres'        
        

class Companies(BaseModel):
    id = IntegerField(unique=True)
    name = TextField()

    class Meta:
        table_name = 'companies'

        
class Countries(BaseModel):
    id = TextField(unique=True)
    name = TextField()

    class Meta:
        table_name = 'countries'


class Movies(BaseModel):
    id = IntegerField(unique=True)
    title = TextField()
    budget = IntegerField(null=True)
    overview = TextField(null=True)
    release_date = DateField(null=True)
    revenue = IntegerField(null=True)
    runtime = IntegerField(null=True)
    tagline = TextField(null=True)
    poster = TextField(null=True)
    vote_average = FloatField()
    vote_count = IntegerField()

    class Meta:
        table_name = 'movies'


class Users(BaseModel):
    id = IntegerField(unique=True)
    username = TextField(unique=True)
    password = TextField()
    email = TextField(unique=True)
    birthday = DateField()
    level = IntegerField()

    class Meta:
        table_name = 'users'


class MoviesGenres(BaseModel):
    movie_id = ForeignKeyField(Movies)
    genre_id = ForeignKeyField(Genres)

    class Meta:
        table_name = 'movies_genres'


class MoviesCompanies(BaseModel):
    movie_id = ForeignKeyField(Movies)
    company_id = ForeignKeyField(Companies)

    class Meta:
        table_name = 'movies_companies'        
        
        
class MoviesCountries(BaseModel):
    movie_id = ForeignKeyField(Movies)
    country_id = ForeignKeyField(Countries)    

    class Meta:
        table_name = 'movies_countries'


class Comments(BaseModel):
    movie_id = ForeignKeyField(Movies)
    user_id = ForeignKeyField(Users)
    content = TextField()
    stars = IntegerField()
    time = DateField()


TABLES = [
    Genres, Companies, Countries, Movies, Users,
    MoviesGenres, MoviesCompanies, MoviesCountries, Comments,
]

#db = sqlite3.connect('movies.db')

with database.connection_context():
    database.create_tables(TABLES, safe=True)
    database.commit()

ImproperlyConfigured: Postgres driver not installed!

In [15]:
#database.close()
print(MoviesGenres)

<Model: MoviesGenres>
